In [ ]:
# 1. Instalación de librerías (Solo si usas Google Colab)
!pip install river geopy folium

# 2. Procesamiento de datos y utilidades
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from geopy.distance import geodesic

# 3. Machine Learning y Stream Learning (River)
from river.forest import ARFClassifier
from river.drift import ADWIN
from river.metrics import Accuracy

# 4. Evaluación de modelos (Scikit-learn)
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# 5. Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from IPython.display import display




In [ ]:
# ==============================
# 1. Carga del dataset
# ==============================

df = pd.read_csv("/content/reportes_guayaquil_balanceado_final.csv")

df["tweet_created"] = pd.to_datetime(
    df["tweet_created"], errors="coerce"
)

df = df.dropna(subset=[
    "tweet_created",
    "latitude",
    "longitude",
    "text"
])

df = df.sort_values("tweet_created").reset_index(drop=True)

print("Total mensajes:", len(df))

In [ ]:
import re

def limpiar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = re.sub(r"http\S+", "", texto)
    texto = re.sub(r"@\w+", "", texto)
    texto = re.sub(r"#", "", texto)
    texto = re.sub(r"[^a-záéíóúñü\s]", " ", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

df["text_clean"] = df["text"].apply(limpiar_texto)


In [ ]:
# ==============================
# 2. Inicialización ARF
# ==============================

arf = ARFClassifier(
    n_models=15,
    max_depth=30,
    lambda_value=6,
    drift_detector=ADWIN(),
    warning_detector=ADWIN(),
    seed=42
)

metric_accuracy = Accuracy()


In [ ]:
# ==============================
# 3. Inicialización DenStream
# ==============================

from river.cluster import DenStream

denstream = DenStream(
    epsilon=0.002,        # ≈ 200 metros (Guayaquil)
    mu=4,
    beta=0.75,
    decaying_factor=0.01,
    n_samples_init=50,
    stream_speed=100
)

In [ ]:
inicio = df["tweet_created"].min()
fin = inicio + pd.Timedelta(hours=1)

df_stream = df[
    (df["tweet_created"] >= inicio) &
    (df["tweet_created"] < fin)
]


In [ ]:
def crear_mapa(df_puntos, titulo):
    if df_puntos.empty:
        print(titulo, "- sin puntos")
        return None

    mapa = folium.Map(
        location=[
            df_puntos["latitude"].mean(),
            df_puntos["longitude"].mean()
        ],
        zoom_start=14
    )

    colores = {
        "accidente": "red",
        "trafico": "blue",
        "obstaculo": "orange"
    }

    for _, row in df_puntos.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,
            color=colores[row["category"]],
            fill=True,
            fill_opacity=0.7,
            popup=row["category"]
        ).add_to(mapa)

    return mapa


In [ ]:
def centro_denso(df_puntos, epsilon_m):
    max_vecinos = -1
    mejor_punto = None

    coords = list(zip(df_puntos["latitude"], df_puntos["longitude"]))

    for i, (lat1, lon1) in enumerate(coords):
        vecinos = 0
        for lat2, lon2 in coords:
            if geodesic((lat1, lon1), (lat2, lon2)).meters <= epsilon_m:
                vecinos += 1

        if vecinos > max_vecinos:
            max_vecinos = vecinos
            mejor_punto = (lat1, lon1)

    return mejor_punto, max_vecinos


In [ ]:
def crear_mapa_con_epsilon_denso(df_puntos, epsilon_m, titulo):
    if df_puntos.empty:
        print(titulo, "- sin puntos")
        return None

    centro, vecinos = centro_denso(df_puntos, epsilon_m)

    if centro is None:
        print("No se pudo determinar centro denso")
        return None

    mapa = folium.Map(
        location=list(centro),
        zoom_start=14
    )

    colores = {
        "accidente": "red",
        "trafico": "blue",
        "obstaculo": "orange"
    }

    # Puntos (eventos)
    for _, row in df_puntos.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,
            color=colores.get(row["category"], "gray"),
            fill=True,
            fill_opacity=0.7
        ).add_to(mapa)

    # Círculo ε REAL
    folium.Circle(
        location=list(centro),
        radius=epsilon_m,
        color="green",
        fill=False,
        popup=f"Centro denso ({vecinos} eventos dentro de ε)"
    ).add_to(mapa)

    return mapa


In [ ]:
# Este es el dataset que entra al stream

print("Dataset base del flujo:")
print(df_stream.shape)

# Verificación rápida

print("\nDistribución de categorías en df_stream:")
print(df_stream["category"].value_counts())

In [ ]:
# =====================================================
# FUNCIÓN: cluster denso por ventana (reutilizable)
# =====================================================
def cluster_denso_por_ventana(puntos, epsilon_m):
    if not puntos:
        return None, 0, None

    df = pd.DataFrame(puntos)
    centro, vecinos = centro_denso(df, epsilon_m)

    if centro is None:
        return None, 0, None

    df_cluster = df[
        df.apply(
            lambda r: geodesic(
                (r["latitude"], r["longitude"]), centro
            ).meters <= epsilon_m,
            axis=1
        )
    ]

    return centro, vecinos, df_cluster


In [ ]:
# =========================================================
# INICIALIZACIÓN
# =========================================================
y_true_ventana = []
y_pred_ventana = []
y_true_total = []
y_pred_total = []
accuracy_evolucion = []
ventanas_tiempo = []
contador_total = 0

clusters_por_ventana = defaultdict(list)
VENTANA_MIN = 10

print("Iniciando procesamiento controlado de 60 minutos...")


# =========================================================
# BUCLE PRINCIPAL
# =========================================================
for minuto, df_min in df_stream.groupby(pd.Grouper(key="tweet_created", freq="1min")):

    if df_min.empty:
        continue

    for _, row in df_min.iterrows():
        texto = row["text_clean"]
        y_real = row["category"]
        lat = row["latitude"]
        lon = row["longitude"]

        x = {"text": texto}
        y_pred = arf.predict_one(x)
        arf.learn_one(x, y_real)

        if y_pred is None:
            y_pred = "unknown"

        y_true_ventana.append(y_real)
        y_pred_ventana.append(y_pred)
        y_true_total.append(y_real)
        y_pred_total.append(y_pred)
        contador_total += 1

        # === SOLO INCIDENTES A DENSTREAM ===
        if y_pred in ["accidente", "trafico", "obstaculo"]:
            denstream.learn_one({"lat": lat, "lon": lon})
            clusters_por_ventana[minuto.minute].append({
                "latitude": lat,
                "longitude": lon,
                "category": y_pred
            })

    # =====================================================
    # REPORTE CADA 10 MINUTOS
    # =====================================================
    min_actual = minuto.minute + 1

    if min_actual % VENTANA_MIN == 0:

        print("\n" + "█"*60)
        print(f" GENERANDO REPORTES: MINUTO {min_actual}")
        print(f" Mensajes en esta ventana: {len(y_true_ventana)} | Total: {contador_total}")
        print("█"*60)

        print("\n--- DenStream ---")
        print(f"Micro-clusters potenciales: {len(denstream.p_micro_clusters)}")
        print(f"Micro-clusters outliers: {len(denstream.o_micro_clusters)}")

        # === MÉTRICAS ARF ===
        acc_v = accuracy_score(y_true_ventana, y_pred_ventana)
        accuracy_evolucion.append(acc_v)
        ventanas_tiempo.append(f"{min_actual}m")

        etiquetas = sorted(list(set(y_true_ventana + y_pred_ventana)))
        cm = confusion_matrix(y_true_ventana, y_pred_ventana, labels=etiquetas)

        plt.figure(figsize=(7, 5))
        sns.heatmap(cm, annot=True, fmt="d",
                    xticklabels=etiquetas,
                    yticklabels=etiquetas,
                    cmap="Blues", cbar=False)
        plt.title(f"Matriz ARF - hasta min {min_actual}")
        plt.xlabel("Predicción")
        plt.ylabel("Real")
        plt.show()

        # === PUNTOS DE LA VENTANA ===
        inicio_v = min_actual - VENTANA_MIN
        puntos_acum = []

        for m in range(inicio_v, min_actual):
            puntos_acum.extend(clusters_por_ventana.get(m, []))

        if puntos_acum:
            df_puntos = pd.DataFrame(puntos_acum)


            # === TABLA CATEGORÍAS ===
            tabla_categorias = (
                df_puntos
                .groupby("category")
                .size()
                .reset_index(name="num_mensajes")
            )

            print("\nDistribución de categorías en la ventana:")
            display(tabla_categorias)

            mapa = crear_mapa(df_puntos, f"Mapa Incidentes (Min {inicio_v}-{min_actual})")
            if mapa:
                display(mapa)
            if puntos_acum:
                df_puntos = pd.DataFrame(puntos_acum)

                mapa_eps = crear_mapa_con_epsilon_denso(
                    df_puntos,
                    epsilon_m=200,  # ajusta a tu epsilon real
                    titulo=f"Mapa con región densa ε (Min {inicio_v}-{min_actual})"
                )

            if mapa_eps:
                    display(mapa_eps)


        # === RESET VENTANA ===
        y_true_ventana = []
        y_pred_ventana = []

print("\n--- PROCESAMIENTO FINALIZADO ---")


In [ ]:
from collections import defaultdict
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from geopy.distance import geodesic
import folium

# =========================================================
# INICIALIZACIÓN
# =========================================================
y_true_ventana = []
y_pred_ventana = []
y_true_total = []
y_pred_total = []
accuracy_evolucion = []
ventanas_tiempo = []
contador_total = 0

clusters_por_ventana = defaultdict(list)
VENTANA_MIN = 10
centro_anterior = None
vecinos_anterior = None
historial_clusters = []

print("Iniciando procesamiento controlado de 60 minutos...")

# =========================================================
# BUCLE PRINCIPAL
# =========================================================
for minuto, df_min in df_stream.groupby(pd.Grouper(key="tweet_created", freq="1min")):

    if df_min.empty:
        continue

    for _, row in df_min.iterrows():
        texto = row["text_clean"]
        y_real = row["category"]
        lat = row["latitude"]
        lon = row["longitude"]

        x = {"text": texto}
        y_pred = arf.predict_one(x)
        arf.learn_one(x, y_real)

        if y_pred is None:
            y_pred = "unknown"

        y_true_ventana.append(y_real)
        y_pred_ventana.append(y_pred)
        y_true_total.append(y_real)
        y_pred_total.append(y_pred)
        contador_total += 1

        # === SOLO INCIDENTES A DENSTREAM ===
        if y_pred in ["accidente", "trafico", "obstaculo"]:
            denstream.learn_one({"lat": lat, "lon": lon})
            clusters_por_ventana[minuto.minute].append({
                "latitude": lat,
                "longitude": lon,
                "category": y_pred
            })

    # =====================================================
    # REPORTE CADA 10 MINUTOS
    # =====================================================
    min_actual = minuto.minute + 1

    if min_actual % VENTANA_MIN == 0:

        print("\n" + "█"*60)
        print(f" GENERANDO REPORTES: MINUTO {min_actual}")
        print(f" Mensajes en esta ventana: {len(y_true_ventana)} | Total: {contador_total}")
        print("█"*60)

        print("\n--- DenStream ---")
        print(f"Micro-clusters potenciales: {len(denstream.p_micro_clusters)}")
        print(f"Micro-clusters outliers: {len(denstream.o_micro_clusters)}")

        # === MÉTRICAS ARF ===
        acc_v = accuracy_score(y_true_ventana, y_pred_ventana)
        accuracy_evolucion.append(acc_v)
        ventanas_tiempo.append(f"{min_actual}m")

        # === PUNTOS DE LA VENTANA ===
        inicio_v = min_actual - VENTANA_MIN
        puntos_acum = []

        for m in range(inicio_v, min_actual):
            puntos_acum.extend(clusters_por_ventana.get(m, []))

        if puntos_acum:
            df_puntos = pd.DataFrame(puntos_acum)

            # ============================================
            # CLUSTER ESPACIAL REAL (usando ε)
            # ============================================
            EPSILON_M = 200

            # === 1. Encontrar centro denso ===
            centro, vecinos = centro_denso(df_puntos, EPSILON_M)
            historial_clusters.append({
                "centro": centro,
                "vecinos": vecinos,
                "ventana_origen": min_actual
            })

            # ============================================
            # COMPARAR CON CLUSTER DE LA VENTANA ANTERIOR
            # ============================================
            puntos_prev = []

            if min_actual > VENTANA_MIN:
                inicio_prev = inicio_v - VENTANA_MIN
                fin_prev = inicio_v

                for m in range(inicio_prev, fin_prev):
                    puntos_prev.extend(clusters_por_ventana.get(m, []))

            centro_prev, vecinos_prev, df_cluster_prev = cluster_denso_por_ventana(
                puntos_prev, EPSILON_M
            )

            if centro_prev:
                print("\n🕒 Cluster ventana anterior:")
                print(f"Centro previo: {centro_prev}")
                print(f"Vecinos previos: {vecinos_prev}")
            else:
                print("\n🕒 No se detectó cluster dominante en la ventana anterior")

            if centro is None:
                print("No se pudo calcular un centro denso.")
            else:
                print(f"\n📍 Centro denso detectado: {centro}")
                print(f"👥 Vecinos dentro de ε: {vecinos}")

                # === 2. Filtrar SOLO puntos dentro del cluster ===
                df_cluster = df_puntos[
                    df_puntos.apply(
                        lambda r: geodesic(
                            (r["latitude"], r["longitude"]), centro
                        ).meters <= EPSILON_M,
                        axis=1
                    )
                ]

                if df_cluster.empty:
                    print("⚠️ No hay puntos dentro del cluster.")
                else:
                    # === 3. Tabla CLUSTER – CATEGORÍA ===
                    tabla_cluster = (
                        df_cluster
                        .groupby("category")
                        .size()
                        .reset_index(name="num_mensajes")
                        .sort_values("num_mensajes", ascending=False)
                    )

                    print("\n📊 TABLA CLUSTER – CATEGORÍA (solo puntos dentro de ε)")
                    display(tabla_cluster)

                    # === 4. Mostrar mapa del CLUSTER (Decay) ===
                    LAMBDA = 0.25
                    UMBRAL_MIN = 5

                    mapa_decay = folium.Map(
                        location=[df_puntos["latitude"].mean(),
                                  df_puntos["longitude"].mean()],
                        zoom_start=13
                    )

                    pesos = []
                    for c in historial_clusters:
                        if c["ventana_origen"] <= min_actual:
                            delta_t = (min_actual - c["ventana_origen"]) / VENTANA_MIN
                            peso = c["vecinos"] * (2 ** (-LAMBDA * delta_t))
                            if peso > UMBRAL_MIN:
                                pesos.append(peso)

                    if pesos:
                        max_peso = max(pesos)
                        for c in historial_clusters:
                            if c["ventana_origen"] > min_actual:
                                continue

                            delta_t = (min_actual - c["ventana_origen"]) / VENTANA_MIN
                            peso = c["vecinos"] * (2 ** (-LAMBDA * delta_t))

                            if peso <= UMBRAL_MIN:
                                continue

                            if c["ventana_origen"] == min_actual:
                                color, etiqueta = "red", "Cluster ACTUAL"
                            elif c["ventana_origen"] == (min_actual - VENTANA_MIN):
                                color, etiqueta = "green", "Cluster ANTERIOR"
                            else:
                                color, etiqueta = "purple", "Cluster en decay"

                            folium.Circle(
                                location=c["centro"],
                                radius=EPSILON_M,
                                color=color,
                                fill=True,
                                fill_opacity=min(0.7, peso / max_peso),
                                tooltip=(
                                    f"{etiqueta} | "
                                    f"Ventana origen: {c['ventana_origen']} min | "
                                    f"Mensajes iniciales: {c['vecinos']} | "
                                    f"Peso actual: {peso:.2f}"
                                )
                            ).add_to(mapa_decay)

                    display(mapa_decay)
                    centro_anterior = centro
                    vecinos_anterior = vecinos

        # === RESET DE VENTANA ===
        y_true_ventana = []
        y_pred_ventana = []

print("\n--- PROCESAMIENTO FINALIZADO ---")

In [ ]:
# =========================================================
# MAPA DE DECAY TEMPORAL – CLUSTERS VIVOS HASTA LA VENTANA
# =========================================================

LAMBDA = 0.25          # mismo decay conceptual
UMBRAL_MIN = 5         # peso mínimo para seguir visualizando
EPSILON_M = 200        # mismo epsilon
minuto_visualizar = min_actual  # usa la ventana actual del loop

mapa_decay = folium.Map(
    location=[df_stream["latitude"].mean(),
              df_stream["longitude"].mean()],
    zoom_start=13
)

# para normalizar opacidad
pesos = []

for c in historial_clusters:
    if c["ventana_origen"] <= minuto_visualizar:
        delta_t = (minuto_visualizar - c["ventana_origen"]) / VENTANA_MIN
        peso = c["vecinos"] * (2 ** (-LAMBDA * delta_t))
        if peso > UMBRAL_MIN:
            pesos.append(peso)

if not pesos:
    print("⚠️ No hay clusters vivos en esta ventana.")
else:
    max_peso = max(pesos)

    for c in historial_clusters:

        if c["ventana_origen"] > minuto_visualizar:
            continue

        delta_t = (minuto_visualizar - c["ventana_origen"]) / VENTANA_MIN
        peso = c["vecinos"] * (2 ** (-LAMBDA * delta_t))

        if peso <= UMBRAL_MIN:
            continue

        # color según antigüedad
        if c["ventana_origen"] == minuto_visualizar:
            color = "red"
            etiqueta = "Cluster ACTUAL"
        elif c["ventana_origen"] == (minuto_visualizar - VENTANA_MIN):
            color = "green"
            etiqueta = "Cluster ANTERIOR"
        else:
            color = "purple"
            etiqueta = "Cluster en decay"

        folium.Circle(
            location=c["centro"],
            radius=EPSILON_M,
            color=color,
            fill=True,
            fill_opacity=min(0.7, peso / max_peso),
            tooltip=(
                f"{etiqueta} | "
                f"Ventana origen: {c['ventana_origen']} min | "
                f"Mensajes iniciales: {c['vecinos']} | "
                f"Peso actual: {peso:.2f}"
            )
        ).add_to(mapa_decay)

    display(mapa_decay)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def mostrar_matriz_visual(y_true, y_pred, titulo, labels):
    """Genera una matriz de confusión limpia para el paper."""
    if not y_true or not y_pred:
        print(f"Sin datos para mostrar en: {titulo}")
        return

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels, yticklabels=labels, cbar=False)
    plt.title(f"{titulo}", fontsize=12)
    plt.ylabel('Clase Real')
    plt.xlabel('Predicción')
    plt.tight_layout()
    plt.show()

In [ ]:
# Gráfico de evolución de Accuracy para el paper
plt.figure(figsize=(10, 5))
plt.plot(ventanas_tiempo, accuracy_evolucion, marker='s', markersize=8, linestyle='-', color='#2c3e50', linewidth=2)
plt.title('Evolución del Desempeño ARF (Curva de Aprendizaje)', fontsize=14)
plt.xlabel('Intervalos de Tiempo', fontsize=12)
plt.ylabel('Accuracy Score', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim(0, 1.05)

# Añadir etiquetas de valor sobre los puntos
for i, val in enumerate(accuracy_evolucion):
    plt.text(i, val + 0.02, f"{val:.2f}", ha='center', fontsize=10, fontweight='bold')

plt.savefig("curva_aprendizaje_final.png", dpi=300, bbox_inches='tight')
plt.show()
from sklearn.metrics import classification_report, accuracy_score

print("\n" + "="*60)
print("REPORTE FINAL CONSOLIDADO (60 MINUTOS - 3000 MENSAJES)")
print("="*60)

# 1. Matriz de Confusión Final
etiquetas_finales = sorted(list(set(y_true_total)))
mostrar_matriz_visual(y_true_total, y_pred_total, "Consolidado Final 60 min", etiquetas_finales)

# Métricas finales consolidadas
print("\nRESUMEN FINAL DE 60 MINUTOS:")
reporte_final = classification_report(y_true_total, y_pred_total, output_dict=True)
df_resumen = pd.DataFrame(reporte_final).transpose()
display(df_resumen.round(3))
df_resumen.to_csv("metricas_finales_reporte.csv")